<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/covid19_tweet/LR_C%3D23~33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [265]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re

warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 36.5MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 72.6MB/s]


In [266]:
train_df.head()

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8


In [267]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [268]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))
train_df.head(2)

,ID,Tweet,Labels,label
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]"
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6]


In [269]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=['0','1','2','3','4','5','6','7','8','9','10'])
onehot[:3]

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [270]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['ID','Labels', 'label'],axis=1)

train_df.head(2)

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [271]:
cols_target = ['0','1','2','3','4','5','6','7','8','9','10']

In [272]:

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [273]:
train_df['Tweet'] = train_df['Tweet'].map(lambda x : clean_text(x))
test_df['Tweet'] = test_df['Tweet'].map(lambda x : clean_text(x))

In [274]:
x = train_df.Tweet
test_x = test_df.Tweet
print(x.shape, test_x.shape)

(5000,) (2500,)


In [275]:
x[0]

'no joke i will hop on a plane rn well after covid19 lol'

In [276]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(max_features=2000, stop_words = 'english')
vector

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [277]:
x_dtm = vector.fit_transform(x)
x_dtm

<5000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 36699 stored elements in Compressed Sparse Row format>

In [278]:
test_x_dtm = vector.fit_transform(test_x)
test_x_dtm

<2500x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 18721 stored elements in Compressed Sparse Row format>

In [279]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=10)

def add_feature(x, feature_to_add) :
    from scipy.sparse import csr_matrix, hstack
    return hstack([x, csr_matrix(feature_to_add).T], 'csr')

In [280]:
weight_0 = {0 : 0.58,
            1 : 0.42} # 대다수 1
weight_1 = {0 : 0.5,
            1 : 0.5} # 비슷
weight_2 = {0 : 0.03, 
            1 : 0.97} # 대다수 0
weight_3 = {0 : 0.12,
            1 : 0.88} # 대다수 0
weight_4 = {0 : 0.17,
            1 : 0.83}  # 대다수 0
weight_5 = {0 : 0.22,
            1 : 0.78} # 대다수 0
weight_6 = {0 : 0.35,
            1 : 0.65} # 대다수 0
weight_7 = {0 : 0.06,
            1 : 0.94} # 대다수 0
weight_8 = {0 : 0.12,
            1 : 0.88} # 대다수 0
weight_9 = {0 : 0.18,
            1 : 0.82} # 대다수 0
weight_10 = {0 : 0.72,
            1 : 0.28} # 대다수 1
weight_list =[weight_0,weight_1,weight_2,weight_3,weight_4
              ,weight_5,weight_6,weight_7,weight_8,weight_9
              ,weight_10]

answer = onehot[:2500]
submission = onehot[:2500].to_numpy()

In [281]:
for weight, label in enumerate(cols_target) :
    print('진행중인 label : {}'.format(label))
    logreg = LogisticRegression(C=23.0, class_weight = weight_list[weight])
    y = train_df[label]

    logreg.fit(x_dtm, y)

    y_pred_x = logreg.predict(x_dtm)
    print('training accuracy : {}'.format(accuracy_score(y, y_pred_x)))

    test_y = logreg.predict(test_x_dtm)
    test_y_prob = logreg.predict_proba(test_x_dtm)[:,1]
    answer[label] = test_y_prob
    answer_p[label] = test_y_prob

    x_dtm = add_feature(x_dtm, y)
    test_x_dtm = add_feature(test_x_dtm, test_y)

진행중인 label : 0
training accuracy : 0.8796
진행중인 label : 1
training accuracy : 0.9784
진행중인 label : 2
training accuracy : 0.9566
진행중인 label : 3
training accuracy : 0.881
진행중인 label : 4
training accuracy : 0.882
진행중인 label : 5
training accuracy : 0.8748
진행중인 label : 6
training accuracy : 0.8808
진행중인 label : 7
training accuracy : 0.9406
진행중인 label : 8
training accuracy : 0.9658
진행중인 label : 9
training accuracy : 0.868
진행중인 label : 10
training accuracy : 0.8694


In [282]:
answer_p = answer.copy()
answer.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.470390,0.028998,0.227417,0.775450,0.024572,0.020165,0.980694,0.130018,0.014861,0.123066,0.147167
1,0.229355,0.006175,0.145260,0.064351,0.049436,0.687429,0.839662,0.033334,0.163568,0.046565,0.041915
2,0.291263,0.005952,0.068628,0.142055,0.556401,0.534650,0.392515,0.024165,0.074941,0.409683,0.095908


In [283]:
answer_p.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.470390,0.028998,0.227417,0.775450,0.024572,0.020165,0.980694,0.130018,0.014861,0.123066,0.147167
1,0.229355,0.006175,0.145260,0.064351,0.049436,0.687429,0.839662,0.033334,0.163568,0.046565,0.041915
2,0.291263,0.005952,0.068628,0.142055,0.556401,0.534650,0.392515,0.024165,0.074941,0.409683,0.095908


In [284]:
answer[answer >= 0.5 ] = 1
answer.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.470390,0.028998,0.227417,1.000000,0.024572,0.020165,1.000000,0.130018,0.014861,0.123066,0.147167
1,0.229355,0.006175,0.145260,0.064351,0.049436,1.000000,1.000000,0.033334,0.163568,0.046565,0.041915
2,0.291263,0.005952,0.068628,0.142055,1.000000,1.000000,0.392515,0.024165,0.074941,0.409683,0.095908


In [285]:
answer_p.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.470390,0.028998,0.227417,0.775450,0.024572,0.020165,0.980694,0.130018,0.014861,0.123066,0.147167
1,0.229355,0.006175,0.145260,0.064351,0.049436,0.687429,0.839662,0.033334,0.163568,0.046565,0.041915
2,0.291263,0.005952,0.068628,0.142055,0.556401,0.534650,0.392515,0.024165,0.074941,0.409683,0.095908


In [286]:
answer[answer < 1] = 0
answer.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [287]:
answer_p.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,0.470390,0.028998,0.227417,0.775450,0.024572,0.020165,0.980694,0.130018,0.014861,0.123066,0.147167
1,0.229355,0.006175,0.145260,0.064351,0.049436,0.687429,0.839662,0.033334,0.163568,0.046565,0.041915
2,0.291263,0.005952,0.068628,0.142055,0.556401,0.534650,0.392515,0.024165,0.074941,0.409683,0.095908


In [288]:
Answer = np.array(answer)
Answer_p = np.array(answer_p)

for row in range(2500) : # 행별로 돌아요
    if Answer_p[row].max() <= 0.5 :
        for idx, value in enumerate(Answer[row]) :
            if value == Answer[row].max() :
                submission[row][idx] = 1

In [289]:
Submission = pd.DataFrame(submission, columns=cols_target)
Submission

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2496,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2498,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
sub = test_df.drop(['Tweet'],axis=1)
sub

,ID
0,5001
1,5002
2,5003
3,5004
4,5005
...,...
2495,7496
2496,7497
2497,7498
2498,7499


In [291]:
sub['Labels'] = 0

In [292]:
for row in range(2500) :
    one_row = []
    for cat in cols_target :
        if int(Submission[row:row+1][cat].values) == 1 :
            one_row.append(cat)
    sub['Labels'][row] = ' '.join(one_row)

In [293]:
from google.colab import files
sub.to_csv('LR_33_weight.csv', index=False)
files.download('LR_33_weight.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>